In [63]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_google_genai langchain_community langchain_core tavily-python wikipedia

In [64]:
from dotenv import load_dotenv

load_dotenv()

True

In [65]:
import os
import json
import requests
from typing import Dict, List, Optional, TypedDict, Annotated, Union
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [66]:
class LeetCodeBuddyState(TypedDict):
    problem_id: str
    problem_statement: Optional[dict]
    editorial: Optional[str]
    problem_summary: Optional[str]
    conversation: Annotated[List[Union[HumanMessage, AIMessage]], add_messages]

In [67]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [ ]:
def html_to_text(html_content: str) -> str:
    """Convert HTML content into readable plain text."""
    soup = BeautifulSoup(html_content, "html.parser")
    text = soup.get_text(separator="\n")  # keep paragraphs and line breaks
    return text.strip()


In [ ]:


def fetch_problem_and_editorial(state: LeetCodeBuddyState) -> LeetCodeBuddyState:
    problem_id = state["problem_id"]
    problem_data = fetch_problem_data(problem_id)
    editorial = fetch_editorial_text(problem_id)
    
    return {
        **state,
        "problem_statement": problem_data,
        "editorial": editorial,
    }


In [ ]:
# Node 2: Summarize the problem
def summarize_problem(state: LeetCodeBuddyState) -> LeetCodeBuddyState:
    problem_data = state["problem_statement"]
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert programming tutor. Summarize the following LeetCode problem in a clear, concise way that highlights the key requirements and constraints."),
        ("human", "Problem: {title}\n\nDescription: {description}\n\nExample: {example}")
    ])
    
    chain = prompt | llm
    summary = chain.invoke({
        "title": problem_data.get("title", ""),
        "description": problem_data.get("description", ""),
        "example": problem_data.get("example", "")
    })
    
    return {
        **state,
        "problem_summary": summary.content.strip(),
        "conversation": [AIMessage(content=f"**Problem Summary:**\n{summary.content}")]
    }